# Motor de búsqueda 

Al ejecutar este código, usuario podrá ingresar el término que desee buscar. Es decir, el algoritmo buscará aquellas acciones PME que mejor se asocien al término ingresado. 

OUTPUT: archivo excel con acciones seleccionadas.

In [1]:
import pandas as pd
import os 
import re
from transformers import pipeline


In [2]:
# Directories [cambiar a directorio propios]
path = r'C:\Users\JoaquinFarina\Dropbox\Team NLP\Data'
# Carpeta con subcarpetas de años con datos PME
pme_path = path + r'\Planes de Mejoramiento Educativo'
os.chdir(pme_path)

In [27]:
# cargar bbdd
df_planificacion_implementacion_total = pd.read_csv('df_planificacion_implementacion_total_indicadores_forma.csv',
                                                    low_memory = False)

In [60]:
# Query
concept = input('¿Qué concepto desea buscar? : ')
agno = input('¿En qué año quienres buscar (2018 - 2022)? : ')

¿Qué concepto desea buscar? Puntualidad
¿En qué año quienres buscar (2018 - 2022)?2019


In [75]:
# Normalizar concepto
concept = unidecode(concept)
concept = re.sub(r'[^\w\s]', ' ', concept)
concept = concept.lower()
# Generar lista de acciones 
df_caso = df_planificacion_implementacion_total[df_planificacion_implementacion_total.agno == int(agno)]
planificacion_descripcion_acciones = list(set(df_caso.descripción_del_plan_planif_norm.values))

In [80]:
# instanciar transformer 
classifier = pipeline("zero-shot-classification", 
                       model="Recognai/bert-base-spanish-wwm-cased-xnli")
# Corriendo sólo para 1000 filas para que prueba sea más rápida 
dict_match = {}
for accion in planificacion_descripcion_acciones[:1000]:
    clasif = classifier(accion, candidate_labels=[concept])
    dict_match[accion] = clasif['scores']

In [81]:
# umbral para filtrar 
umbral = 0.5
# Generar dataframe con scores 
df_resumen = pd.DataFrame(dict_match).T.reset_index()
df_resumen.columns = ['descripción_del_plan_planif_norm',concept]
df_resumen = df_resumen.sort_values(by=[concept], ascending=False)
df_resumen= df_resumen[df_resumen[concept]>=umbral]
# Consolidar con resto de los datos, pero sólo para acciones seleccionadas
df_final = pd.merge(df_caso, df_resumen, on = 'descripción_del_plan_planif_norm', how='right')
# Guardar
df_final.to_excel('bsuqueda_en_acciones_pme_' + concept + '_' + agno + '.xlsx')